In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Preparation

In [2]:
# loading
def load_csv(filename):
    # file_loc = 'files/'+filename+'.csv'
    file_loc = filename
    data = pd.read_csv(file_loc, low_memory=False)
    return data

# create count table
def calculate_count_table(df,column_name):
    counts = df[column_name].value_counts().reset_index()
    counts.columns = [column_name, 'count']
    
    counts = counts.sort_values(by='count')
    return counts

def calculate_count_table_with_null(df,column_name):
    counts = df[column_name].value_counts().reset_index()
    counts.columns = [column_name, 'count']
    counts = counts.sort_values(by='count')
    
    null_count = df[column_name].isnull().sum()
    null_row = pd.DataFrame({column_name: ['null'], 'count': [null_count]})
    counts = pd.concat([counts, null_row], ignore_index=True)
    
    return counts



### Code

In [3]:
filename="export.csv"

raw_data = load_csv(filename)
df = raw_data.copy() 
users=df.drop(columns=['trackable_id','trackable_type','trackable_name','trackable_value','checkin_date']).copy().drop_duplicates()

In [4]:
df

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
0,QEVuQwEABlEzkh7fsBBjEe26RyIVcg==,NaN,NaN,NaN,2015-11-26,1069,Condition,Ulcerative colitis,0
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,32.0,male,US,2015-11-26,1069,Condition,Ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3168,Condition,pain in left upper arm felt like i was getting...,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3169,Condition,hip pain when gettin up,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3170,Condition,pain in hand joints,4
...,...,...,...,...,...,...,...,...,...
7976218,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,1,Tag,tired,NaN
7976219,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,2,Tag,stressed,NaN
7976220,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,9002,Food,soup,NaN
7976221,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,9139,Food,yogurt,NaN


## Symptom

In [15]:
df_symptoms = df.loc[(df['trackable_type']=="Symptom")]

In [16]:
pd.DataFrame(df_symptoms)

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
15,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,123,Symptom,Joint stiffness,3
24,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-26,152,Symptom,Nausea,1
31,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,152,Symptom,Nausea,2
32,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,8,Symptom,Anxiety,2
33,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,242,Symptom,Fatigue,1
...,...,...,...,...,...,...,...,...,...
7976213,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,3368,Symptom,difficulty getting up,4
7976214,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,153,Symptom,Neck pain,2
7976215,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,242,Symptom,Fatigue,3
7976216,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,1026,Symptom,Poor concentration,3


In [35]:
import emoji
import re
from nltk.corpus import stopwords
import string
import unicodedata
import contractions
import inflect
from nltk.tokenize import word_tokenize


def emojis_words(text):
    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))
    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")
    return clean_text

def clean_text(input_text):    
    
    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)
    
    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    # Важно понимать эмоциональную окраску обрабатываемого текста
    clean_text = emojis_words(clean_text)
    
    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    # Преобразование символов с диакритическими знаками к ASCII-символам: используем функцию normalize из модуля unicodedata и преобразуем символы с диакритическими знаками к ASCII-символам
    clean_text = unicodedata.normalize('NFKD', clean_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Разворачиваем сокращения: текст часто содержит конструкции вроде "don't" или "won't", поэтому развернём подобные сокращения
    clean_text = contractions.fix(clean_text)

    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(temp.number_to_words(word))
        else:
            words.append(word)
    clean_text = ' '.join(words)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text

In [49]:
# df_symptoms['trackable_name'] = df_symptoms['trackable_name'].apply(clean_text)

In [69]:
symptom_table_count = calculate_count_table(df_symptoms,"trackable_name")
df_sy = symptom_table_count.sort_values(by='count', ascending=False).head(30)
df_sy.rename(columns={'trackable_name': 'symptom', 'count': 'symptom count'}, inplace=True)

In [42]:
df_symptoms.to_csv('df_symptoms.csv', encoding='utf-8')

### Condition

In [50]:
df_condition = df.loc[(df['trackable_type']=="Condition")]
df_condition

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
0,QEVuQwEABlEzkh7fsBBjEe26RyIVcg==,NaN,NaN,NaN,2015-11-26,1069,Condition,Ulcerative colitis,0
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,32.0,male,US,2015-11-26,1069,Condition,Ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3168,Condition,pain in left upper arm felt like i was getting...,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3169,Condition,hip pain when gettin up,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3170,Condition,pain in hand joints,4
...,...,...,...,...,...,...,...,...,...
7976184,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-12-04,7,Condition,Achalasia,3
7976206,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,2075,Condition,Period pain,1
7976207,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,64,Condition,Anxiety,3
7976208,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,291,Condition,Depression,2


In [51]:
df_condition['trackable_name'] = df_condition['trackable_name'].apply(clean_text)
df_condition.to_csv('df_condition.csv', encoding='utf-8')
df_condition

<ipython-input-51-34c88cdd61d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['trackable_name'] = df_condition['trackable_name'].apply(clean_text)


,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
0,QEVuQwEABlEzkh7fsBBjEe26RyIVcg==,NaN,NaN,NaN,2015-11-26,1069,Condition,ulcerative colitis,0
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,32.0,male,US,2015-11-26,1069,Condition,ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3168,Condition,pain left upper arm felt like getting shot,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3169,Condition,hip pain gettin,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,2.0,female,CA,2017-04-28,3170,Condition,pain hand joints,4
...,...,...,...,...,...,...,...,...,...
7976184,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-12-04,7,Condition,achalasia,3
7976206,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,2075,Condition,period pain,1
7976207,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,64,Condition,anxiety,3
7976208,QEVuQwEAtlfm8VyoxZ9biWjDHb74gQ==,22.0,female,GB,2019-12-04,291,Condition,depression,2


In [70]:
condition_table_count = calculate_count_table(df_condition,"trackable_name")
df_co = condition_table_count.sort_values(by='count', ascending=False).head(30)
df_co.rename(columns={'trackable_name': 'condition', 'count': 'condition count'}, inplace=True)

### Treatment

In [52]:
df_treatment = df.loc[(df['trackable_type']=="Treatment")]
df_treatment

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
25,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-26,9890,Treatment,Zofran,8.0 mg
34,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,9890,Treatment,Zofran,4.0 mg
35,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,4934,Treatment,Klonopin,0.5 mg
201,QEVuQwEA08uRNqb7wltBMKWvJGIUwA==,37.0,female,US,2015-05-26,1470,Treatment,Caffeine,100.0 mg
202,QEVuQwEA08uRNqb7wltBMKWvJGIUwA==,37.0,female,US,2015-05-26,5681,Treatment,Methadone,20.0 mg
...,...,...,...,...,...,...,...,...,...
7976180,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-03,9951,Treatment,Mesalazine,1g
7976181,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-13,9951,Treatment,Mesalazine,1g
7976182,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-21,9951,Treatment,Mesalazine,1g
7976196,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-12-04,9951,Treatment,Mesalazine,1g


In [53]:
df_treatment['trackable_name'] = df_treatment['trackable_name'].apply(clean_text)
df_treatment.to_csv('df_treatment.csv', encoding='utf-8')
df_treatment

<ipython-input-53-ae2cbc620fdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treatment['trackable_name'] = df_treatment['trackable_name'].apply(clean_text)


,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
25,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-26,9890,Treatment,zofran,8.0 mg
34,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,9890,Treatment,zofran,4.0 mg
35,QEVuQwEAO+R1md5HUn8+w1Qpbg7ogw==,NaN,NaN,NaN,2015-05-27,4934,Treatment,klonopin,0.5 mg
201,QEVuQwEA08uRNqb7wltBMKWvJGIUwA==,37.0,female,US,2015-05-26,1470,Treatment,caffeine,100.0 mg
202,QEVuQwEA08uRNqb7wltBMKWvJGIUwA==,37.0,female,US,2015-05-26,5681,Treatment,methadone,20.0 mg
...,...,...,...,...,...,...,...,...,...
7976180,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-03,9951,Treatment,mesalazine,1g
7976181,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-13,9951,Treatment,mesalazine,1g
7976182,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-10-21,9951,Treatment,mesalazine,1g
7976196,QEVuQwEA/3ZnJPHdtLiOWBy1VAYIYA==,48.0,male,GB,2019-12-04,9951,Treatment,mesalazine,1g


In [67]:
treatment_table_count = calculate_count_table(df_treatment,"trackable_name")
df_tr = treatment_table_count.sort_values(by='count', ascending=False).head(30)
df_tr.rename(columns={'trackable_name': 'treatment', 'count': 'treatment count'}, inplace=True)

In [71]:
merged_df = pd.concat([df_tr, df_sy, df_co], axis=1)
merged_df

,treatment,treatment count,symptom,symptom count,condition,condition count
0,ibuprofen,21485,headache,108719,fibromyalgia,55447
1,magnesium,11424,fatigue,107800,depression,50122
2,vitamin d3,11379,nausea,89624,anxiety,46968
3,paracetamol,11253,brain fog,73226,chronic fatigue syndrome,28262
4,vitamin,10577,joint pain,65443,migraine,26116
5,gabapentin,9627,fatigue tiredness,63395,ibs,17328
6,tramadol,9284,anxiety,61610,fatigue,14920
7,prednisone,8309,diarrhea,52425,asthma,14219
8,naproxen,8211,dizziness,50162,endometriosis,13907
9,multivitamin,7925,depression,43485,ehlersdanlos syndrome,13679
